<a href="https://colab.research.google.com/github/427Stanley427/Python/blob/main/G%C3%A9n%C3%A9rer%20une%20paire%20de%20cl%C3%A9%20bitcoin%20avec%20prefix%20et%20balance%20positive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bitcoin

In [ ]:
pip install requests

In [ ]:
import requests
import bitcoin

def generate_bitcoin_keypair_with_prefix(prefix):
    while True:
        private_key = bitcoin.random_key()
        public_key_uncompressed = bitcoin.privtopub(private_key)
        address_uncompressed = bitcoin.pubtoaddr(public_key_uncompressed)

        if address_uncompressed.startswith(prefix):
            public_key_compressed = bitcoin.compress(bitcoin.decode_pubkey(public_key_uncompressed, 'hex'))
            address_compressed = bitcoin.pubtoaddr(public_key_compressed)

            wif_private_key = bitcoin.encode_privkey(bitcoin.decode_privkey(private_key, 'hex'), 'wif')

            balance_uncompressed = get_balance(address_uncompressed)
            balance_compressed = get_balance(address_compressed)

            if balance_uncompressed > 0 or balance_compressed > 0:
                return {
                    "private_key_hex": private_key,
                    "wif_private_key": wif_private_key,
                    "public_key_uncompressed": public_key_uncompressed,
                    "public_key_compressed": public_key_compressed,
                    "address_uncompressed": address_uncompressed,
                    "address_compressed": address_compressed,
                    "balance_uncompressed": balance_uncompressed,
                    "balance_compressed": balance_compressed
                }

def get_balance(address):
    url = f"https://api.blockchair.com/bitcoin/dashboards/address/{address}"
    response = requests.get(url)
    data = response.json().get("data", {})

    if not data:
        return 0

    address_data = data.get(address, {})
    if not address_data:
        return 0

    balance_satoshis = address_data.get("address", {}).get("balance", 0)
    return balance_satoshis / 100_000_000  # Convertir en BTC

kp = generate_bitcoin_keypair_with_prefix("1")

print("Clé privée hex :", kp["private_key_hex"])
print("Clé privée WIF :", kp["wif_private_key"])
print("Adresse non compressée :", kp["address_uncompressed"], "| Balance:", kp["balance_uncompressed"], "BTC")
print("Adresse compressée :", kp["address_compressed"], "| Balance:", kp["balance_compressed"], "BTC")
